# Excel Sheet Front-End

https://github.com/myliang/x-spreadsheet

## Libraries

In [1]:
from httpserver import start, stop
from http.server import SimpleHTTPRequestHandler, HTTPServer
import os
from urllib.parse import urlparse, parse_qs
import ipywidgets as widgets
import time

## Backend: the HTTP server

### Widgets python implementation

In [2]:
logsw=widgets.Textarea(layout=widgets.Layout(width='50%'))
logsw.value = ""
bag ={}
class myHTTPRequestHandler(SimpleHTTPRequestHandler):
    callback={}
    def __init__(self, *args, directory=None,bag=bag, **kwargs):
        self.bag = bag
        self.directory = os.path.join(os.getcwd(),"widgets")
        super().__init__(*args, directory=self.directory, **kwargs)
        # print(self.directory)
    def end_headers(self):
        super().end_headers()
    def do_GET(self):
        self.parsed_path = urlparse(self.path)
        self.queryparams = parse_qs(self.parsed_path.query)
        if self.path.endswith('/version'):
            self.version()
        elif self.parsed_path.path.endswith('/setvalue'):
            self.setvalue()
        else:
            super().do_GET()
    def version(self):
        ans = '{"version": "0.0"}'
        eans = ans.encode()        
        self.send_response(200)
        self.send_header("Content-type", "application/json")
        self.send_header("Content-Length",len(eans))
        self.end_headers()
        self.wfile.write(eans)
    def setvalue(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['value'][0]})
        if self.queryparams['variable'][0] in myHTTPRequestHandler.callback:
            self.callback[self.queryparams['variable'][0]](self.queryparams['value'][0])
        self.version() 
    def log_message(self, format, *args):
        global logsw
        v = format % args
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        logsw.value += current_time + " " + v +"\n"
logsw

Textarea(value='', layout=Layout(width='50%'))

In [3]:
start(handler_class=myHTTPRequestHandler, port=8085)

Starting httpd...



## Frontend

### Widget javascript implementation

In [4]:
class LoadModule(object):
    """This class is loading the xlsx.min.js module"""
    def _repr_javascript_(self):
        return '''debugger;
var id = document.getElementById("x_spreadsheet")
if (id!=null){
    if ('x_spreadsheet' in window){
        // the module main.js is already loaded
        fetch(window.location.origin+'/proxy/8085/setvalue?value=1&variable=loaded')
    } else {
        console.log("makeIt not found!!!!")
    }
} else {
    const fileref = document.createElement('link');
    fileref.rel = "stylesheet";
    fileref.type = "text/css";
    fileref.href = "https://unpkg.com/x-data-spreadsheet@1.1.5/dist/xspreadsheet.css";
    document.head.append(fileref);
    var script= document.createElement('script');
    script.src = "https://unpkg.com/x-data-spreadsheet@1.1.5/dist/xspreadsheet.js";
    script.type = "module"
    script.id = "x_spreadsheet"
    document.head.appendChild(script);
    function check(){
       if (!('x_spreadsheet' in window)){
           setTimeout(function(){ check()}, 500);
       }
    }
    check()
    // sub modules are still not loaded (let them 2 secs)
    setTimeout(fetch, 2000, window.location.origin+'/proxy/8085/setvalue?value=1&variable=loaded' )
}
'''

In [5]:
class ExecuteModule(object):
    """This class execute the code once the modul is loaded"""
    def _repr_javascript_(self):
        return '''debugger;
x_spreadsheet('#xspreadsheet');'''

In [6]:
class DisplayModule(object):
    """This class is creating the element where the output of the will set UI elements"""
    def _repr_javascript_(self):
        return '''debugger;
        let divWrapper = document.createElement('div');
        divWrapper.id = "xspreadsheet";
        element.appendChild(divWrapper);        
        '''    

In [7]:
DisplayModule()

In [8]:
def loaded_cb(value):
    display(ExecuteModule())
myHTTPRequestHandler.callback.update({"loaded": loaded_cb})
LoadModule()

In [9]:
#stop()